In [85]:
import numpy as np
import gym
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
#from tensorflow.keras.optimizers import RMSprop
from keras import optimizers

from rl.agents.dqn import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import EpsGreedyQPolicy,LinearAnnealedPolicy, BoltzmannQPolicy
from rl.core import Processor

In [86]:
def create_agent(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape = (1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [87]:
class TaxiProcessor(Processor):
    def process_observation(self, observation):
        one_hot = np.zeros(500)
        one_hot[observation] = 1
        return one_hot

    def process_reward(self, reward):
        return reward

In [88]:
env = gym.make("Taxi-v3")
print("No of state = ", env.observation_space.n)
print("No of actions =", env.action_space.n)
model = create_agent(500, env.action_space.n)
model.summary()

No of state =  500
No of actions = 6
Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_12 (Flatten)        (None, 500)               0         
                                                                 
 dense_48 (Dense)            (None, 24)                12024     
                                                                 
 dense_49 (Dense)            (None, 24)                600       
                                                                 
 dense_50 (Dense)            (None, 24)                600       
                                                                 
 dense_51 (Dense)            (None, 6)                 150       
                                                                 
Total params: 13,374
Trainable params: 13,374
Non-trainable params: 0
_________________________________________________________________


In [89]:
memory = SequentialMemory(limit=80000, window_length=1)
processor = TaxiProcessor()

policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.05, value_test=.005, nb_steps=10000)

dqn = DQNAgent(model=model, nb_actions=env.action_space.n, processor=processor, memory=memory,
               nb_steps_warmup=1000, gamma=0.99, policy=policy, enable_double_dqn=False, target_model_update=1e-3)

In [90]:
optimizer = tf.keras.optimizers.legacy.RMSprop(lr=1e-3)
#optimizer1 = optimizers.Adam(lr=1e-3)
dqn.compile(optimizer, metrics=['mae'])
dqn.fit(env, nb_steps=100000, visualize=False, verbose=1)

C:\Users\jackt\anaconda3\lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Training for 100000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 78s 8ms/step - reward: -2.5783
50 episodes - episode_reward: -513.080 [-875.000, -194.000] - loss: 0.278 - mae: 5.396 - mean_q: -3.367 - mean_eps: 0.478 - prob: 1.000

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 92s 9ms/step - reward: -1.1764
52 episodes - episode_reward: -228.135 [-623.000, -1.000] - loss: 0.236 - mae: 10.756 - mean_q: -9.637 - mean_eps: 0.050 - prob: 1.000

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 91s 9ms/step - reward: -1.1509
57 episodes - episode_reward: -202.018 [-578.000, 7.000] - loss: 0.583 - mae: 14.398 - mean_q: -12.859 - mean_eps: 0.050 - prob: 1.000

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 91s 9ms/step - reward: -1.1038
64 episodes - episode_reward: -169.250 [-398.000, 10.000] - loss: 0.865 - mae: 16.182 - mean_q: -12.294 - mean_ep

In [91]:
env.reset()
res = dqn.test(env, nb_episodes=100, visualize=False)


Testing for 100 episodes ...
Episode 1: reward: 2.000, steps: 19
Episode 2: reward: 5.000, steps: 16
Episode 3: reward: 13.000, steps: 8
Episode 4: reward: 9.000, steps: 12
Episode 5: reward: 6.000, steps: 15
Episode 6: reward: 2.000, steps: 19
Episode 7: reward: 7.000, steps: 14
Episode 8: reward: 10.000, steps: 11
Episode 9: reward: 5.000, steps: 16
Episode 10: reward: 5.000, steps: 16
Episode 11: reward: 4.000, steps: 17
Episode 12: reward: -200.000, steps: 200
Episode 13: reward: 8.000, steps: 13
Episode 14: reward: 9.000, steps: 12
Episode 15: reward: 5.000, steps: 16
Episode 16: reward: 10.000, steps: 11
Episode 17: reward: 5.000, steps: 16
Episode 18: reward: 2.000, steps: 19
Episode 19: reward: 10.000, steps: 11
Episode 20: reward: 0.000, steps: 21
Episode 21: reward: 6.000, steps: 15
Episode 22: reward: 5.000, steps: 16
Episode 23: reward: 4.000, steps: 17
Episode 24: reward: 3.000, steps: 18
Episode 25: reward: 8.000, steps: 13
Episode 26: reward: 11.000, steps: 10
Episode 27

In [92]:
res.history
print(np.average(res.history['episode_reward']))

-17.87
